In [32]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt  
import matplotlib.cbook as cbook
import numpy as np
import os
import sys
import requests
import json
import datawash


In [56]:
AKEY = 'Pt15VZ8Msk0fjKO4SXLUD58CiwbOOZ0P'
def get_coord(address):
    """从百度地图API获取地址对应的坐标。
    """
    response = requests.get('http://api.map.baidu.com/geocoder/v2/?address=' + address 
                            + '&ak=' + AKEY + '&output=json')
    address_dict = json.loads(response.text)
    return address_dict

def get_map():
    """从百度地图API获取地图并存储
    """
    width = '1024'
    height = '768'
    centerlng = 105.0
    centerlat = 37.0
    response = requests.get('http://api.map.baidu.com/staticimage/v2?ak=' + AKEY
                            + '&width='+ width + '&height='+ height
                            + '&zoom=5&center=' + str(centerlng)+','+str(centerlat))
    with open('map.png','wb') as img:
        img.write(response.content)
    edge = {'left':67.177, 'right':142.606,'top':56.269,'bottom':11.467}
    return os.path.abspath('map.png'),edge

In [38]:
# 遍历所有用户，读取需要的信息
locationslist = list()

jsons = datawash.datajsons()
for user in jsons:
    try:
        locationslist.append(user['locations'][0]['name'])
    except:
        pass

In [14]:
# 批量获取坐标
coord_dict = dict()
templist = list(set(locationslist))
for loc in templist:
    coord_dict[loc] = get_coord(loc)

In [17]:
print(len(coord_dict))

16589


In [16]:
print(coord_dict)

{'L.A': {'status': 1, 'msg': 'Internal Service Error:无相关结果', 'results': []}, '校长家': {'status': 1, 'msg': 'Internal Service Error:无相关结果', 'results': []}, '魔法之森的雾雨魔法店': {'status': 0, 'result': {'location': {'lng': 113.91779000614618, 'lat': 22.54995386851273}, 'precise': 0, 'confidence': 80, 'level': '购物'}}, 'New York -> Seattle': {'status': 0, 'result': {'location': {'lng': 117.36730743253084, 'lat': 32.92746633167288}, 'precise': 1, 'confidence': 80, 'level': '餐饮'}}, 'Beijing': {'status': 0, 'result': {'location': {'lng': 117.56987361914868, 'lat': 34.8567448967647}, 'precise': 1, 'confidence': 80, 'level': '购物'}}, '四川绵阳': {'status': 0, 'result': {'location': {'lng': 102.89915972360397, 'lat': 30.36748093795755}, 'precise': 0, 'confidence': 0, 'level': '省'}}, 'Gainesville': {'status': 1, 'msg': 'Internal Service Error:无相关结果', 'results': []}, 'Maryland': {'status': 0, 'result': {'location': {'lng': 120.63216948246401, 'lat': 31.31622818981102}, 'precise': 1, 'confidence': 80, 'level': '

In [115]:
# 获取地图
img = get_map()
imgfile = img[0]
imgedge = img[1]
# 绘制地图
image_file = cbook.get_sample_data(imgfile)  
image = plt.imread(image_file)
fig, ax= plt.subplots(1,1)
ax.imshow(image)
# 绘制用户分布
# 'lng':data['result']['location']['lng']
# 'lat':data['result']['location']['lat']
x = list()
y = list()

d = [11.467, 22.195, 32.429, 41.558, 49.36, 56.269]
for loc in locationslist:
    if coord_dict[loc]['status'] == 0:
        lng = coord_dict[loc]['result']['location']['lng']
        lat = coord_dict[loc]['result']['location']['lat']
        tempx =((lng-imgedge['left'])/(imgedge['right']-imgedge['left'])) * 1024
#         tempy = ((imgedge['top']-lat)/(imgedge['top']-imgedge['bottom'])) * 768
        for i in range(len(d)-1):
            if lat > d[i] and lat <= d[i+1]:
                tempy = (((d[i+1]-lat)/(d[i+1]-d[i]))/5.0+(1-(i+1)/5.0))*768
        x.append(tempx)
        y.append(tempy)

x = np.array(x)
y = np.array(y)
ax.scatter(x, y, color='#FF8C00',alpha=0.1, edgecolors='none')
ax.axis('off')
fig.show()

In [89]:
fig, ax= plt.subplots(1,1)
ax.plot(np.array([11.476,22.195,32.429,41.558,49.36,56.269]))
fig.show()